<a href="https://colab.research.google.com/github/sarasabino/Accident_Rate_Project/blob/main/Src/Notebooks/00_Data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Generation

Although we have the data from the accidents occurred at the company we need to have the data from the staff with its characteristics to be able to develop a machine learning algorithm. 

The idea is to generate fake data to be able to perform the algorithm and then when data is real we can apply it.

For that purpose we are going to generate a csv file with the staff of the company. 

We are going to assume this is the static company's staff.

In [228]:
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/HS_Accidentabilidad.csv .
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/'G_Plantas y Tech'.csv .
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/Datos_plantilla_fabrica.csv .




We are going to upload the accidents data we have

In [229]:
import pandas as pd

accidentes = pd.read_csv('HS_Accidentabilidad.csv', sep=';')

accidentes = accidentes[(accidentes['Año']== 2018) | (accidentes['Año']==2019) | (accidentes['Año']==2020) ]

In [230]:
len(accidentes)

4180

In [231]:
accidentes['Gravedad potencial']

2057    Muy grave
2062         Leve
2063         Leve
2064         Leve
2065         Leve
          ...    
7200          NaN
7430          NaN
7436          NaN
7580          NaN
7581          NaN
Name: Gravedad potencial, Length: 4180, dtype: object

In [232]:
staff = accidentes[['Codigo accidentado', 'Puesto habitual de trabajo','Turno','Edad','Género','Planta', 'Tipo accidente', 'Tipo trabajador', 'Gravedad potencial', 'Lugar o seccion accidente']]
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente
2057,HS-CLMB_2546,Operario de Puente Grúa,Turno de mañana,31,Hombre,20,Accidente c/b,ETT,Muy grave,Células Soldadura
2062,HS-CEPA_2563,Engraving,Turno de mañana,35,Hombre,19,Accidente c/b,Indefinido,Leve,Tubos
2063,HS-GNVT_2564,Titular L-8,Turno de tarde,54,Hombre,12,Accidente s/b,Duración Determinada,Leve,Tubos
2064,HS-GNVT_2565,Carretillero,Turno de mañana,NaN,Hombre,12,Incidente,ETT,Leve,Almacén de Producto Terminado
2065,HS-DNGG_2566,NaN,Turno de mañana,NaN,NaN,41,Incidente,Incidente (no aplica),Leve,Blanking


In [233]:
planta = pd.read_csv('G_Plantas y Tech.csv', sep=';')
planta.drop(columns={'Activo'}, inplace=True)
planta.head()

,ID,Fabrica,Tecnología,Pais
0,1,Argentina,Blanking,Argentina
1,2,Argentina,Corte Longitudinal,Argentina
2,3,Argentina,Corte Transversal,Argentina
3,4,Arizona,Células Soldadura,EEUU
4,5,Arizona,Corte Láser,EEUU


In [234]:
len(staff)

4180

In [235]:
staff = staff.merge(planta, left_on='Planta', right_on='ID', how='left')


In [236]:
staff.drop(columns={'ID'}, inplace=True)

staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais
0,HS-CLMB_2546,Operario de Puente Grúa,Turno de mañana,31,Hombre,20,Accidente c/b,ETT,Muy grave,Células Soldadura,Burgos,Skinpass,España
1,HS-CEPA_2563,Engraving,Turno de mañana,35,Hombre,19,Accidente c/b,Indefinido,Leve,Tubos,Burgos,Recocido,España
2,HS-GNVT_2564,Titular L-8,Turno de tarde,54,Hombre,12,Accidente s/b,Duración Determinada,Leve,Tubos,Barcelona,Skinpass,España
3,HS-GNVT_2565,Carretillero,Turno de mañana,NaN,Hombre,12,Incidente,ETT,Leve,Almacén de Producto Terminado,Barcelona,Skinpass,España
4,HS-DNGG_2566,NaN,Turno de mañana,NaN,NaN,41,Incidente,Incidente (no aplica),Leve,Blanking,Hiasa,Corte Láser,España


In [237]:
len(staff)

4180

Now we are going to upload some staff data to get the number of trainning hours per plant and have a reference of the quantity of empleados we want to generate. This data is aggregated so it is not valid for our analysis

In [238]:
rrhh_plant = pd.read_csv('Datos_plantilla_fabrica.csv', sep=';')

rrhh_plant['Año'].unique()

array([2017., 2016., 2018., 2019.,   nan, 2020., 2021.])

In [239]:
# we are getting only 2020 for our analysis

rrhh_plant = rrhh_plant[rrhh_plant['Año']==2020]


In [240]:
rrhh_plant.head()

,ID,Planta,Año,Mes,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata,Horas Presencia Efectiva Propios,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Horas Formacion General Propios,Empleados Propios - indefinido,Empleados Propios - duración determinada
2615,3819,Barcelona,2020.0,ENE,203.0,28.0,8.0,20.0,29770.30,3138.0,1307.5,15.0,23.0,211.0,189.0,14.0
2616,3820,Burgos,2020.0,ENE,239.0,68.0,28.0,40.0,32625.00,6487.0,4549.3,445.0,348.5,757.0,188.0,38.0
2617,3821,Tarragona,2020.0,ENE,49.0,17.0,5.0,12.0,8352.50,2080.0,862.0,8.0,12.0,111.0,46.0,3.0
2618,3822,Valencia,2020.0,ENE,153.0,28.0,24.0,3.0,22105.20,647.0,3968.3,8.0,6.0,35.5,139.0,2.0
2619,3823,Portugal,2020.0,ENE,99.0,12.0,10.0,2.0,16625.75,384.0,1866.5,80.0,32.5,514.0,72.0,28.0


In [241]:
trainning_hours = rrhh_plant[['Planta','Horas Presencia Efectiva Subcontratados', 'Horas Presencia Efectiva ETTs', 'Horas Formacion Seguridad Propios', 'Horas Formacion Seguridad ETTs']]

In [242]:
trainning_hours.rename(columns={'Planta':'Fabrica'}, inplace=True)
trainning_hours.head(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Fabrica,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
2615,Barcelona,3138.0,1307.5,15.0,23.0


In [243]:
trainning_hours = trainning_hours.groupby('Fabrica').agg('sum').reset_index()

In [244]:
staff = staff.merge(trainning_hours, on='Fabrica', how='left')
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
0,HS-CLMB_2546,Operario de Puente Grúa,Turno de mañana,31,Hombre,20,Accidente c/b,ETT,Muy grave,Células Soldadura,Burgos,Skinpass,España,25249.0,42484.0,1419.3,830.1
1,HS-CEPA_2563,Engraving,Turno de mañana,35,Hombre,19,Accidente c/b,Indefinido,Leve,Tubos,Burgos,Recocido,España,25249.0,42484.0,1419.3,830.1
2,HS-GNVT_2564,Titular L-8,Turno de tarde,54,Hombre,12,Accidente s/b,Duración Determinada,Leve,Tubos,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5
3,HS-GNVT_2565,Carretillero,Turno de mañana,NaN,Hombre,12,Incidente,ETT,Leve,Almacén de Producto Terminado,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5
4,HS-DNGG_2566,NaN,Turno de mañana,NaN,NaN,41,Incidente,Incidente (no aplica),Leve,Blanking,Hiasa,Corte Láser,España,0.0,167848.0,1300.5,101.5


Now we are going to calculate the proportion in 2020 between people and accidents occur

In [245]:
employees = pd.read_csv('Datos_plantilla_fabrica.csv', sep=';')
employees = employees[['Año', 'Empleados Propios', 'Empleados Ajenos', 'Empleados ETTS',  'Empleados Subcontrata']]
employees = employees.fillna(0)
employees = employees.groupby(['Año']).agg('sum')
employees.head()

,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata
Año,,,,
0.0,15754.0,3277.0,2490.0,786.0
2016.0,40609.0,9160.0,0.0,0.0
2017.0,43291.0,9794.0,0.0,0.0
2018.0,49865.0,10973.0,7964.0,2993.0
2019.0,50625.0,10527.0,7823.0,2697.0


In [246]:
accidentes = pd.read_csv('HS_Accidentabilidad.csv', sep=';')
accidentes = accidentes[['Año', 'Codigo accidentado']]
accidentes = accidentes.groupby('Año').agg('count').reset_index()
accidentes.rename(columns={'Codigo accidentado':'Accidentes'}, inplace=True)
accidentes.head()

,Año,Accidentes
0,2015.0,202
1,2016.0,1335
2,2017.0,1203
3,2018.0,1532
4,2019.0,1526


In [247]:
employees = employees.merge(accidentes, on='Año', how='inner')
employees.head()

,Año,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata,Accidentes
0,2016.0,40609.0,9160.0,0.0,0.0,1335
1,2017.0,43291.0,9794.0,0.0,0.0,1203
2,2018.0,49865.0,10973.0,7964.0,2993.0,1532
3,2019.0,50625.0,10527.0,7823.0,2697.0,1526
4,2020.0,65644.0,7114.0,5773.0,1335.0,1122


In [248]:
employees.dtypes

Año                      float64
Empleados Propios        float64
Empleados Ajenos         float64
Empleados ETTS           float64
Empleados Subcontrata    float64
Accidentes                 int64
dtype: object

In [249]:
employees['Total_emp'] = employees['Empleados Propios'] + employees['Empleados Ajenos'] + employees['Empleados ETTS'] + employees['Empleados Subcontrata']

In [250]:
employees = employees[['Año', 'Accidentes', 'Total_emp']]

employees['perc_acc'] = employees['Accidentes'] / employees['Total_emp'] * 100
employees.head()

,Año,Accidentes,Total_emp,perc_acc
0,2016.0,1335,49769.0,2.682393
1,2017.0,1203,53085.0,2.266177
2,2018.0,1532,71795.0,2.133853
3,2019.0,1526,71672.0,2.129144
4,2020.0,1122,79866.0,1.404853


For our analysis we are going to suppose we have a year staff, with all people. 
We would have 79900 people with around 7000 people who have suffered an accident on the past years.

In [251]:
len(staff)

4180

In [252]:
#we add a new column to indetify those people who have suffered an accident in our dataset
staff['Accident'] = 'Yes'
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident
0,HS-CLMB_2546,Operario de Puente Grúa,Turno de mañana,31,Hombre,20,Accidente c/b,ETT,Muy grave,Células Soldadura,Burgos,Skinpass,España,25249.0,42484.0,1419.3,830.1,Yes
1,HS-CEPA_2563,Engraving,Turno de mañana,35,Hombre,19,Accidente c/b,Indefinido,Leve,Tubos,Burgos,Recocido,España,25249.0,42484.0,1419.3,830.1,Yes
2,HS-GNVT_2564,Titular L-8,Turno de tarde,54,Hombre,12,Accidente s/b,Duración Determinada,Leve,Tubos,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5,Yes
3,HS-GNVT_2565,Carretillero,Turno de mañana,NaN,Hombre,12,Incidente,ETT,Leve,Almacén de Producto Terminado,Barcelona,Skinpass,España,25684.5,18302.3,1150.6,151.5,Yes
4,HS-DNGG_2566,NaN,Turno de mañana,NaN,NaN,41,Incidente,Incidente (no aplica),Leve,Blanking,Hiasa,Corte Láser,España,0.0,167848.0,1300.5,101.5,Yes


In [253]:
#plant data we dont have any null values
staff['Planta'].unique()

array([20, 19, 12, 41,  7,  1, 13, 28,  2,  5, 26, 16, 25, 15, 36, 35, 33,
       53, 54,  4, 14, 34, 27, 29, 32, 57,  6, 56, 30, 55, 46, 43, 45, 18,
       44, 63, 47, 84])

In [254]:
##work position

positions = staff['Puesto habitual de trabajo'].unique()
len(positions)

794

In [255]:
import random
import numpy as np 

staff['wk_position'] = np.random.choice(positions, size=len(staff))

In [256]:
staff['wk_position'].unique()

array(['operative Logistikfachkraft', 'Saumschrott durchtrennen',
       'Bediener Biele-Wender, Halle 8', 'Gruista/soldador',
       'Línea de Prensa - Cargador de Bobina', 'OPERARIO DE MÁQUINA',
       'Bestücker LSL 2', 'OPERARIO LONGI 05', 'Auxiliar/Montador',
       'Operario de montacarga', 'LINEA DE PRENSA', 'press op ',
       'Operador puente grúa', 'carretilla', 'Proceso Galva',
       'Mantenimiento', 'Operario de linea', 'tooling work',
       'UTILLERO GALVANIZADO', 'Montador/auxiliar',
       'Operador Paletizadora', 'PRENSA002', 'Empaquetador perfiladoras',
       '包装packaging', 'Carreteillero', 'AYUDANTE PRENSA002',
       'Maquinista tubos', 'Jigging', 'Zona de Picos de Nave 3',
       'Entrada Decapado', 'Gruista Nave 2 - 3',
       'Press operator Ln 3  2nd shift',
       'Ayudante de perfiladora de Dimeco', 'GRUISTA PTA07 PTA08',
       'Titular LONGI001', 'Kranfahrer', 'Instandhalter',
       'Cargador descargador', 'Auxiliar/montador LONGI002',
       'Maquinista 

In [257]:
staff.drop(columns={'Puesto habitual de trabajo'}, inplace=True)

In [258]:
#gender

staff.rename(columns={'Género':'Gender'}, inplace=True)


In [260]:
staff['Gender'].unique()

array(['Hombre', nan, 'Mujer'], dtype=object)

In [275]:
staff['Gender'] = staff['Gender'].fillna('Hombre')

In [276]:
len(staff[staff['Gender'].isnull()])

0

In [277]:
staff['Gender'].unique()

array(['Hombre', 'Mujer'], dtype=object)